# MCMC4.0: Replica Exchange Monte Carlo (Parallel Tempering)

In order to get correct and unbiased results from QMC simulations, we need to implement a "global updating" algorithm in addition to the single-flip "local updating." Here I try to rewrite the previous code using Channels to implement parallelization to show how it goes in Jupyter Notebook. However, in most cases ResumableFunctions.jl + MPI.jl is much faster, and you should use MPI.jl instead of using Channels. The implementation based on MPI.jl will be included in mcmc4.5.jl.

## Autocorrelation

It was already found (especially in MCMC3.5) that the autocorrelation length is a key feature determining the efficiency of MCMC. StatsBase.jl includes methods for the autocorrelation.

In [1]:
using Statistics
using StatsBase

In order to decrease the autocorrelation length, we basically need to increase the acceptance rate.

In the case of continuous variables, the acceptance rate can be increased by using a Gibbs sampler (100%) or a hybrid Monte Carlo (almost 100%). You can easily check the autocorrelation of the Gibbs sampler.

However, those algorithms usually do not work for discrete variables, which are important for physical applications. Another way to reduce the autocorrelation length is by implementing global updating. There are various technologies for global updating, but replica exchange is the most universal (in the same way as Metropolis-Hastings was universal for any probabilistic distributions).

## Replica exchange

Last but not least, we forgot mentioning that we can simply increase the acceptance rate by changing the proposed distribution. This point will be discussed in MCMC6.5.

## Kitaev model revisited

There are two ways to exchange the distributions, one by exchanging the temperature, the other by exchanging the internal states. Apparently, the former is faster, but in the case of the Kitaev model the latter is still acceptable because the internal state is just labbeled by the flux sector and not memory-consumptive. I will implement by exchaging the states. *There is another reason for this decision, which you will see later.* First, let me rewrite the previous code by BitVector.

Then, let's rewrite the code by Channels!

Unfortunately, Chennel itself is not distributed over processes but works serially (currently in Julia 1.0). In order to make it parallel, we need to use RemoteChannel instead.

RemoteChannel is not iterable, so partially I used Channel to sample from each distribution. This is why I exchanged the states instead of the temperatures. Processes are interacting via RemoteChannel and sampling is done inside each process using Channel.